In [ ]:
from IPython.display import clear_output
import random

#Username and passwords for login function
userPass = {"amontanus":"Python24", "Apple7":"Gamer12"}


#Formatting for text
class color:
    BOLD = "\033[1m"
    END = "\033[0m"
    BASE = "\033[0m"
    RED = "\u001b[38;2;219;29;29m"
    GREEN  = "\u001b[32m"
    ORANGE  = "\u001b[38;5;202m"
    PURPLE  = "\u001b[38;2;144;9;222m"
    BLUE = "\u001b[38;2;21;86;237m"
    LIME = "\u001b[38;2;173;224;50m"

#Boat colors for each boat
boatColors = {color.ORANGE: "Carrier", color.PURPLE: "Battleship", color.GREEN: "Destroyer", color.BLUE: "Submarine", color.LIME: "Patrol"}

#Initialize boat status dictionaries used while running the game
boatStatus = {boat: False for boat in boatColors.values()}
enBoatStatus = {boat: False for boat in boatColors.values()}


def login():
    '''
    Function that creates a login with a username and password
    Arg(s): none
    Returns: True or False based on if the username and password are correct
    '''
    while True:
        inUser = input("Enter your username: ")
        if inUser == " ":
            return False
        elif inUser in userPass.keys():
            inPass = input("Enter your password: ")
            if inPass == userPass.get(inUser):
                return True
        else:
            print("Incorrect, try again")


def makeBoard():
    '''
    Function that creates the boards
    Arg(s): none
    Returns: the board
    '''
    return [
    ["  ", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J"],
    [" 1", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 2", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 3", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 4", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 5", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 6", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 7", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 8", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    [" 9", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
    ["10", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"]]


def printBoard():
    '''
    Function that prints out the boards
    Arg(s): none
    Returns: the boards printed out
    '''
    clear_output()
    print(color.BOLD + 'Your Board: \n' + color.END)
    for r in playerBoatMap:
        for c in r:
            print(c+color.BASE,end = " ")
        print()
    print("\n \n")
    print(color.BOLD + 'Hit Board: \n' + color.END)
    for r in hitBoard:
        for c in r:
            print(c+color.BASE,end = " ")
        print()
    print("\n \n")


def validLocat(placeBoard, bName, bLen):
    '''
    Function that converts the inputted location to work internally
    Arg(s): board with the boats, boat name, and boat length
    Returns: the converted x and y coordinates
    '''
    while True:
        boatLocat = (input(f"Select {bName} location [{bLen} blocks]: ")).upper()
        try:
            boatY = ord(boatLocat[0]) - 64
            boatX = int(boatLocat[1:])
            if 0 < boatX < 11 and 0 < boatY < 11:
                possiLocat = 0
                directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Directions: right, left, down, up
                for dx, dy in directions:
                    checkArea = [placeBoard[boatX + dx * i][boatY + dy * i] for i in range(bLen) if 0 < (boatX + dx * i) < 11 and 0 < (boatY + dy * i) < 11]
                    if all(item == "-" for item in checkArea):
                        possiLocat += 1
                if possiLocat >= 1:
                    return boatX, boatY
                else:
                    print("Boat overlaps with existing boats, try again")
            else:
                print("Invalid location, try again")
        except:
            print("Invalid location format, try again")


def validRot(placeBoard, boatX, boatY, bLen, bColor):
    '''
    Function that checks to see if a rotation is valid
    Arg(s): board with the boats, boat positions (x and y), boat length, and boat color
    Returns: the board with the boat in an appropriate orientation
    '''
    while True:
        userRot = input(f"Enter the direction for the boat [R = Right, L = Left, U = Up, D = Down]: ").upper()
        
        if userRot == "R" and boatY + bLen <= 11 and all(placeBoard[boatX][boatY+i] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX][boatY + i] = bColor + "■"
            return placeBoard
        if userRot == "L" and boatY - bLen >= 0 and all(placeBoard[boatX][boatY-i] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX][boatY - i] = bColor + "■"
            return placeBoard
        if userRot == "U" and boatX - bLen >= 0 and all(placeBoard[boatX-i][boatY] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX - i][boatY] = bColor + "■"
            return placeBoard
        if userRot == "D" and boatX + bLen <= 11 and all(placeBoard[boatX+i][boatY] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX + i][boatY] = bColor + "■"
            return placeBoard
        else:
            print("Invalid rotation")


def setBoat(bName, bLen, bColor, playerBoatMap):
    '''
    Function that uses previous functions to place and rotate the boat
    Arg(s): boat name, boat length, boat color, and board with boats
    Returns: the board with the boat in the proper location and orientation
    '''
    boatX, boatY = validLocat(playerBoatMap, bName, bLen)
    playerBoatMap[boatX][boatY] = bColor+"■"
    printBoard()
    playerBoatMap = validRot(playerBoatMap, boatX, boatY, bLen, bColor)
    printBoard()


def setEnemyBoat(bLen, bColor, placeBoard):
    '''
    Function that sets up the enemy boat map with all the boats
    Arg(s): boat length, boat color, and board with boats
    Returns: the board with all the boats in the proper location and orientation
    '''
    while True:
        boatX, boatY = random.randint(1, 10), random.randint(1, 10)
        if 0 < boatX < 11 and 0 < boatY < 11:
            directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Directions: right, left, down, up
            possiLocat = sum(all(placeBoard[boatX + dx * i][boatY + dy * i] == "-" for i in range(bLen) if 0 < (boatX + dx * i) < 11 and 0 < (boatY + dy * i) < 11) for dx, dy in directions)
            if possiLocat >= 1:
                break
    while True:
        directs = ["R", "L", "U", "D"]
        userRot = random.choice(directs)
        
        if userRot == "R" and boatY + bLen <= 11 and all(placeBoard[boatX][boatY+i] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX][boatY + i] = bColor + "■"
            break
        if userRot == "L" and boatY - bLen >= 0 and all(placeBoard[boatX][boatY-i] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX][boatY - i] = bColor + "■"
            break
        if userRot == "U" and boatX - bLen >= 0 and all(placeBoard[boatX-i][boatY] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX - i][boatY] = bColor + "■"
            break
        if userRot == "D" and boatX + bLen <= 11 and all(placeBoard[boatX+i][boatY] == "-" for i in range(1, bLen)):
            for i in range(bLen):
                placeBoard[boatX + i][boatY] = bColor + "■"
            break


def objCount(placeBoard, objCheck):
    '''
    Function that counts the amount of a user specified objects in a specific board    
    Arg(s): the board to check on and the object to check for
    Returns: the count of those objects on the specific board
    '''
    count = 0
    for row_index, row in enumerate(placeBoard):
        for col_index, cell in enumerate(row):
            if cell == objCheck:
                count += 1
    return count


def hitCheck (enemyBoatMap, hitBoard):
    '''
    Function that validates the users hit and then hits on the board   
    Arg(s): the enemy map and the player hit board
    Returns: the enemy board with the hit/miss
    '''
    while True: 
        boatLocat = (input("Select a hit location: ")).upper()
        try:
            boatY = ord(boatLocat[0]) - 64
            boatX = int(boatLocat[1:])
            if 0 < boatX < 11 and 0 < boatY < 11:
                if enemyBoatMap[boatX][boatY] == "■" or enemyBoatMap[boatX][boatY] == color.RED + "■":
                    print("You've already hit there, try again")
                elif enemyBoatMap[boatX][boatY] == "-":
                    enemyBoatMap[boatX][boatY] = "■"
                    hitBoard[boatX][boatY] = "■"
                    return "You Missed"
                else:
                    enemyBoatMap[boatX][boatY] = color.RED + "■"
                    hitBoard[boatX][boatY] = color.RED + "■"
                    return "You Hit"
        except:
            print("Invalid format")


def enemyHit(playerBoatMap):
    '''
    Function that has the computer pick a random hit location
    Arg(s): the player board
    Returns: the player board with a hit/miss at the choosen location
    '''
    while True:
        boatX, boatY = random.randint(1, 10), random.randint(1, 10)
        if playerBoatMap[boatX][boatY] == "-":
            playerBoatMap[boatX][boatY] = "■"
            return "Computer Missed"
        elif playerBoatMap[boatX][boatY] != "■" and playerBoatMap[boatX][boatY] != color.RED + "■":
            playerBoatMap[boatX][boatY] = color.RED + "■"
            return "Computer Hit"
            

#Login function to stop the code from running until complete
#login()

hitBoard = makeBoard() # Board for players hit locations
playerBoatMap = makeBoard() # Board with specific boats for player
enemyBoatMap = makeBoard() # Board with specific boats for enemy

printBoard()

#Placing in the player boats
setBoat("Carrier", 5, color.ORANGE, playerBoatMap)
setBoat("Battleship", 4, color.PURPLE, playerBoatMap)
setBoat("Destroyer", 3, color.GREEN, playerBoatMap)
setBoat("Submarine", 3, color.BLUE, playerBoatMap)
setBoat("Patrol", 2, color.LIME, playerBoatMap)

#Placing in the enemy boats
setEnemyBoat(5, color.ORANGE, enemyBoatMap)
setEnemyBoat(4, color.PURPLE, enemyBoatMap)
setEnemyBoat(3, color.GREEN, enemyBoatMap)
setEnemyBoat(3, color.BLUE, enemyBoatMap)
setEnemyBoat(2, color.LIME, enemyBoatMap)

printBoard()

#Game loop
while True:
    #Messages for any boats which the player has destroyed
    playerMsg = hitCheck(enemyBoatMap, hitBoard)
    printBoard()
    #Messages for any boats the computer has destroyed
    enemyMsg = enemyHit(playerBoatMap)
    printBoard()
    print(playerMsg)
    print(enemyMsg)

    #Check if any of the player boats have been destroyed
    for boat_color, boat_name in boatColors.items():
        if not boatStatus[boat_name] and (objCount(playerBoatMap, boat_color + "■") == 0):
            print(f"Your {boat_name} has been destroyed")
            boatStatus[boat_name] = True

    #If all of the player boats have been destroyed, then end the game
    if all(boatStatus.values()):
        print("\nAll of your boats have been destroyed, you lose!\n")
        break

    #Check if any of the enemy boats have been destroyed
    for boat_color, boat_name in boatColors.items():
        if not enBoatStatus[boat_name] and (objCount(enemyBoatMap, boat_color + "■") == 0):
            print(f"Enemy {boat_name} has been destroyed")
            enBoatStatus[boat_name] = True

    #If all of the enemy boats have been destroyed, then end the game
    if all(enBoatStatus.values()):
        print("\nAll of enemy's boats have been destroyed, you win!\n")
        break


'''
To-do (maybe):
- generalize the placing functions
- remove flickering of board
- remove fast inputs crashing the code
- improve resource managment
'''

Your Board: 

   A B C D E F G H I J 
 1 - - - - - - - - - - 
 2 - - - - - - - - - - 
 3 - - - - - - - - - - 
 4 - - - - - - - - - - 
 5 - - - - - - - - - - 
 6 - - - - - - - - - - 
 7 - - - - - - - - - - 
 8 - - - - - - - - - - 
 9 - - - - - - - - - - 
10 - - - - - - - - - - 

 

Hit Board: 

   A B C D E F G H I J 
 1 - - - - - - - - - - 
 2 - - - - - - - - - - 
 3 - - - - - - - - - - 
 4 - - - - - - - - - - 
 5 - - - - - - - - - - 
 6 - - - - - - - - - - 
 7 - - - - - - - - - - 
 8 - - - - - - - - - - 
 9 - - - - - - - - - - 
10 - - - - - - - - - - 

 

